# Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Data Preperation

### Load Dataset

In [2]:
# Import dataset
dataset = pd.read_csv("./data/train.csv")

In [3]:
dataset.dtypes

label       int64
pixel0      int64
pixel1      int64
pixel2      int64
pixel3      int64
            ...  
pixel779    int64
pixel780    int64
pixel781    int64
pixel782    int64
pixel783    int64
Length: 785, dtype: object

In [4]:
dataset.shape

(42000, 785)

## Preprocess the dataset

In [5]:
# divide preliminary dataset set into features and labels
X = dataset.drop(columns=["label"]) # features
y = dataset['label'] # labels

### From Grayscale to RGB

In [6]:
def grey_to_RGB(data:pd.core.frame.DataFrame, IMG_RES:int):
    dim = np.zeros((IMG_RES,IMG_RES)) # Dimension Array
    X_RGB = [] # Empty list to save RGB images 
    X_temp = data.drop(columns=["label"])
    for i in range(X.shape[0]):
        im = X_temp.iloc[i].to_numpy() # fetch image
        im = im.reshape(IMG_RES,IMG_RES) # Reshape the image
        R = np.stack((im,dim,dim),axis=2) # Add Red Dimension
        G = np.stack((dim,im,dim),axis=2) # Add Green Dimension
        B = np.stack((dim,dim,im),axis=2) # Add Blue Dimension
        X_RGB.append(R+G+B)
    return X_RGB # Returns the List of Images

In [7]:
# Convert 1D Image to RGB
X_RGB = grey_to_RGB(dataset,28)

In [8]:
print("Number of rows in dataset are: ",len(X_RGB))
print(X_RGB[0].shape)
print(type(X_RGB[0]))
print(X_RGB[0].dtype)

Number of rows in dataset are:  42000
(28, 28, 3)
<class 'numpy.ndarray'>
float64


### Reshape the Dataset

In [9]:
import cv2 
def format_image(images:list, IMAGE_RES:int):
    reshaped_images = []
    for image in images:
        image = cv2.resize(image, (IMAGE_RES, IMAGE_RES))
        reshaped_images.append(image)
    return reshaped_images

In [10]:
# Resize RGB Images from 28x28 to 224x224
IMAGE_RES = 224
X_RGB = format_image(X_RGB,IMAGE_RES)

In [11]:
print("Number of rows in dataset are: ",len(X_RGB))
print("Shape of each element in dataset is: ",X_RGB[0].shape)
print(type(X_RGB[0]))
print(X_RGB[0].dtype)

Number of rows in dataset are:  42000
Shape of each element in dataset is:  (224, 224, 3)
<class 'numpy.ndarray'>
float64


### Scale the Input Pixels


In [14]:
def scale_pixels(images:list):
    reshaped_images = []
    for image in images:
        image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
        #image = tf.keras.applications.vgg16.preprocess_input(image)
        reshaped_images.append(image)
    return reshaped_images

In [15]:
X_scaled = scale_pixels(X_RGB)

In [16]:
print("Number of rows in dataset are: ",len(X_scaled))
print("Shape of each element in dataset is: ",X_scaled[0].shape)
print(type(X_scaled[0]))
print(X_scaled[0].dtype)

Number of rows in dataset are:  42000
Shape of each element in dataset is:  (224, 224, 3)
<class 'numpy.ndarray'>
float64


### Change dtype of images

In [ ]:
def change_dtype(images:list,dtype:np.dtype=np.float32):
    reshaped_images = []
    for image in images:
        image = np.array(image,dtype=dtype)
        reshaped_images.append(image)
    return reshaped_images

In [ ]:
deneme = np.array(X_scaled[0],dtype=np.float32)

In [ ]:
print("Number of rows in dataset are: ",len(X_scaled))
print("Shape of each element in dataset is: ",deneme.shape)
print(type(deneme))
print(deneme.dtype)

### Split the dataset

In [17]:
from sklearn.model_selection import train_test_split
# split the X_RGB into Training and Test sets  
X_train, X_test, y_train, y_test = train_test_split(X_RGB,y,test_size=0.8, stratify=y) # 80 to 20% randomly

In [ ]:
deneme = change_dtype(X_train)

# Modelling

### Import the Pre-Trained Model

In [18]:
vgg16_model = tf.keras.applications.vgg16.VGG16()
mobilenet_model = tf.keras.applications.MobileNetV2()

In [19]:
mobilenet_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

### Setup the Layers

In [21]:
from tensorflow.keras import layers
model = tf.keras.Sequential()

for layer in mobilenet_model.layers[:-1]: # this is where I changed your code
    model.add(layer)    

# Freeze the layers 
for layer in model.layers:
    layer.trainable = False

# Add 'softmax' instead of earlier 'prediction' layer.
model.add(layers.Dense(10, activation='softmax'))

ValueError: Exception encountered when calling layer "block_2_add" (type Add).

A merge layer should be called on a list of inputs. Received: inputs=Tensor("Placeholder:0", shape=(None, 56, 56, 24), dtype=float32) (not a list of tensors)

Call arguments received by layer "block_2_add" (type Add):
  • inputs=tf.Tensor(shape=(None, 56, 56, 24), dtype=float32)

In [ ]:
model.summary()

### Set the model parameters

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
EPOCHS = 2 
BATCH_SIZE = 32

### Train the model

In [ ]:
from math import ceil
history = model.fit(X_train, y_train,
                    epochs=EPOCHS,
                    shuffle=False,
                    batch_size=BATCH_SIZE
                    )

### Evaluate the model

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(10,7))
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
results = model.predict(X_test)

print(results.shape)

print(results[-1].argmax())
print(type(results[-1].argmax()))

print(y_test.shape[0])
print(y_test.iloc[-1])
print(type(y_test.iloc[-1]))
print(type(y_test))

### Predict the results

In [ ]:
y_predict = model.predict(X_test)

### Evaluate the test accuracy

In [ ]:
def acc_model(y_test, y_predict):
    if y_test.shape[0] != y_predict.shape[0]:
        print("Invalid Input, size of the arrays do not match!")
        return 0
    else:
        sum=0
        for i in range(y_test.shape[0]):
            if y_test.iloc[i] == y_predict[i].argmax():
                sum += 1
    return float("%.4f" % (sum/y_test.shape[0]))

In [ ]:
accuracy = acc_model(y_test,y_predict)
accuracy